In [0]:
from __future__ import print_function
from torchvision.datasets import utils
import torch.utils.data as data_utils
import torch
import os
import numpy as np
from torch import nn, optim
from torch.nn.modules import upsampling
from torch.functional import F
from torch.optim import Adam
from torchvision.utils import save_image
from torch.autograd import Variable


BATCH_SIZE=128

def get_data_loader(dataset_location, batch_size):
    URL = "http://www.cs.toronto.edu/~larocheh/public/datasets/binarized_mnist/"
    # start processing
    def lines_to_np_array(lines):
        return np.array([[int(i) for i in line.split()] for line in lines])
    splitdata = []
    for splitname in ["train", "valid", "test"]:
        filename = "binarized_mnist_%s.amat" % splitname
        filepath = os.path.join(dataset_location, filename)
        utils.download_url(URL + filename, dataset_location)
        with open(filepath) as f:
            lines = f.readlines()
        x = lines_to_np_array(lines).astype('float32')
        x = x.reshape(x.shape[0], 1, 28, 28)
        # pytorch data loader
        dataset = data_utils.TensorDataset(torch.from_numpy(x))
        dataset_loader = data_utils.DataLoader(x, batch_size=batch_size, shuffle=splitname == "train")
        splitdata.append(dataset_loader)
    return splitdata
  
train_loader, valid_loader, test_loader = get_data_loader("binarized_mnist", BATCH_SIZE)
  
#torch.manual_seed(42)
torch.cuda.manual_seed(42)
device = torch.device("cuda" )


print(torch.__version__)




  0%|          | 0/78400000 [00:00<?, ?it/s]

78405632it [00:08, 9067184.59it/s]                              
  0%|          | 0/15680000 [00:00<?, ?it/s]

15687680it [00:02, 7473919.39it/s]                              
  0%|          | 0/15680000 [00:00<?, ?it/s]

15687680it [00:02, 5831053.22it/s]                              


1.0.1.post2


In [0]:
epochs=20
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.e1 = nn.Conv2d(1, 32, kernel_size=3)
        self.e2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.e3 = nn.Conv2d(32, 64, kernel_size=3)
        self.e4 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.e5 = nn.Conv2d(64, 256, kernel_size=5)
        
        self.e61 = nn.Linear(256, 100)
        self.e62= nn.Linear(256,100)
        
        self.d1= nn.Linear(100,256)
        self.conv1= nn.Conv2d(256, 64, 5, padding=4)
        self.upsample= nn.UpsamplingBilinear2d(scale_factor=2)
        self.conv2= nn.Conv2d(64, 32, kernel_size=3, padding=2)
        
        self.conv3= nn.Conv2d(32, 16, kernel_size=3, padding= 2)
       
        self.conv4= nn.Conv2d(16,1 , kernel_size=3, padding=2)
       
        
    def encoder(self, x):
   
        h1= F.elu(self.e1(x))
        h3= F.elu( self.e3(self.e2(h1)) )
        h5= F.elu(self.e5(self.e4(h3)))
      
        h6=h5.view(-1, 256)
        h61= self.e61(h6)
        h62= self.e62(h6)
        
        return h61, h62
        #returning the mean and log variance
        
    def reparametrize(self, mu , log_variance):
       # print('original size mu: ', mu.size())
        sigma= torch.exp(0.5 * log_variance)
        e= torch.randn_like(sigma)
        z= mu+ e*sigma
        return z
    
    #K samples. Sends K Z valyes, E samples
    def reparametrize_iwae(self,K, mu , log_variance):
       
        sigma= torch.exp(0.5 * log_variance)
        z_samples= []
        e_samples=[]
        #generating K samples for the IWAE
        for i in range(K):
          e= torch.randn_like(sigma)        
          z= mu+ e*sigma
 
          z_samples.append(z)
          e_samples.append(e) 
        
        return torch.stack(z_samples),mu,sigma, torch.stack(e_samples)
    
    
    def decoder(self, z):
        
        h1= F.elu(self.d1(z))
        
        h1= h1.view(h1.size(0), 256, 1, 1)
             
        h2= F.elu(self.conv1(h1))
        
        h3=self.upsample(h2)
      
        h4= F.elu(self.conv2(h3))
        h5= self.upsample(h4)
      
        h6= F.elu(self.conv3(h5))
      
        h7= self.conv4(h6)
      
        h10= F.sigmoid(h7)
        
        return h10

    def forward(self, x):
        mu, log_variance= self.encoder(x)
        z= self.reparametrize(mu,log_variance)

        return self.decoder(z), mu, log_variance
    
    #evaluate within one mini-batch
    def evaluate_IWAE_within_one_mini_batch(self,K, inputs,x_tilda, mu, log_variance):
        
        z_samples, mu_h1, sigma, eps_samples = self.reparametrize_iwae(K, mu , log_variance)
  
        log_Qz_x = torch.sum(-0.5*(eps_samples)**2 - torch.log(sigma), -1)       
        #K,128. first dimension basically contains how many samples taken
        
        p_set=[]
        for z in z_samples:      
          p = self.decoder(z)
          p_set.append(p)
        p_x_given_z=torch.stack(p_set)
        
        log_P_z = torch.sum(-0.5*z_samples**2, -1)
        #K, 128
              
        cross_entropy = F.binary_cross_entropy(x_tilda, inputs, reduction='none')
        cross_entropy= cross_entropy.sum(-1).sum(-1).sum(-1)
        
        #doing normalisation
        log_weight = log_P_z - cross_entropy - log_Qz_x
        log_weight = log_weight - torch.max(log_weight, 0)[0]
        weight = torch.exp(log_weight)
        weight = weight / torch.sum(weight, 0)
        weight = Variable(weight.data, requires_grad = False)
        
        loss = (torch.sum(weight * (log_P_z - cross_entropy - log_Qz_x) , 0))
       # print('loss dimension is: ', loss.size())
        return loss
    
    
    
    
    #some inspiration taken from other codes
    #Evaluation with IWAE, across the whole dataset
    def evaluate_IWAE(self,K, inputs,x_tilda, mu, log_variance):
        
        z_samples, mu_h1, sigma, eps_samples = self.reparametrize_iwae(K, mu , log_variance)
  
        log_Qz_x = torch.sum(-0.5*(eps_samples)**2 - torch.log(sigma), -1)       
        #K,128. first dimension basically contains how many samples taken
        
        p_set=[]
        for z in z_samples:      
          p = self.decoder(z)
          p_set.append(p)
        p_x_given_z=torch.stack(p_set)
        
        log_P_z = torch.sum(-0.5*z_samples**2, -1)
        #K, 128
              
        cross_entropy = F.binary_cross_entropy(x_tilda, inputs, reduction='none')
        cross_entropy= cross_entropy.sum(-1).sum(-1).sum(-1)
        
        #doing normalisation
        log_weight = log_P_z - cross_entropy - log_Qz_x
        log_weight = log_weight - torch.max(log_weight, 0)[0]
        weight = torch.exp(log_weight)
        weight = weight / torch.sum(weight, 0)
        weight = Variable(weight.data, requires_grad = False)
        
        loss = torch.sum(torch.sum(weight * (log_P_z - cross_entropy - log_Qz_x) , 0))
        return loss      
    
      
      
model = VAE().to(device)
optimiser= optim.Adam(model.parameters(), lr=3e-4)

def loss_function(x_tilda, x, mu, log_variance):
  #  print('size of x tilda:',x_tilda.size(), ':x size: ', x.size() )
    cross_entropy = F.binary_cross_entropy(x_tilda, x, reduction='sum')
    #print('cross entropy: ', cross_entropy)
    #cross_entropy/= BATCH_SIZE    

    Kl_divergence = -0.5 * torch.sum(1 + log_variance - mu.pow(2) - log_variance.exp())
    
    #Kl_divergence/= BATCH_SIZE
    
    #print('cross entropy: ', cross_entropy,' : KL_divergence: ',Kl_divergence )
    return cross_entropy + Kl_divergence

  
#some reference taken from the mentioned code in slack  
def train(epoch):
    model.train()
    training_loss = 0
    for batch_index, data in enumerate(train_loader):
        data = data.to(device)
        optimiser.zero_grad()
        reconstruction_batch, mu, log_variance = model(data)
          
        loss = loss_function(reconstruction_batch, data, mu, log_variance)
        loss.backward()
        training_loss = training_loss + loss.item()
        optimiser.step()
        if batch_index % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_index * len(data), len(train_loader.dataset),
                100. * batch_index / len(train_loader),
                -loss.item()/len(data) ))
      
      
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, -training_loss / len(train_loader.dataset)))

    

def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            data = data.to(device)
            reconstruction_batch, mu, log_variance = model(data)
            test_loss += loss_function(reconstruction_batch, data, mu, log_variance).item()
            
    test_loss /= len(test_loader.dataset)
   
    print('====> Test set loss: {:.4f}'.format(-test_loss ))

def test_validation(epoch):
    model.eval()
    validation_loss = 0
    with torch.no_grad():
        for i, data in enumerate(valid_loader):
            data = data.to(device)
            reconstruction_batch, mu, log_variance = model(data)
            validation_loss += loss_function(reconstruction_batch, data, mu, log_variance).item()
            
    validation_loss /= len(valid_loader.dataset)
   
    print('====> Validation set loss: {:.4f}'.format(-validation_loss ))
    
    
    
if __name__ == "__main__":
    for epoch in range(1, epochs + 1):
        train(epoch)
        test(epoch)
        test_validation(epoch)
     

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.UpsamplingBilinear2d is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/50000 (0%)]	Loss: -552.321655
Train Epoch: 1 [12800/50000 (26%)]	Loss: -212.512329
Train Epoch: 1 [25600/50000 (51%)]	Loss: -171.057739
Train Epoch: 1 [38400/50000 (77%)]	Loss: -167.097916
====> Epoch: 1 Average loss: -211.6868
====> Test set loss: -154.1568
====> Validation set loss: -155.3712
Train Epoch: 2 [0/50000 (0%)]	Loss: -156.700912
Train Epoch: 2 [12800/50000 (26%)]	Loss: -145.893539
Train Epoch: 2 [25600/50000 (51%)]	Loss: -141.245819
Train Epoch: 2 [38400/50000 (77%)]	Loss: -133.530609
====> Epoch: 2 Average loss: -141.0354
====> Test set loss: -128.7647
====> Validation set loss: -130.1685
Train Epoch: 3 [0/50000 (0%)]	Loss: -129.216614
Train Epoch: 3 [12800/50000 (26%)]	Loss: -127.551834
Train Epoch: 3 [25600/50000 (51%)]	Loss: -120.235840
Train Epoch: 3 [38400/50000 (77%)]	Loss: -115.707817
====> Epoch: 3 Average loss: -123.4553
====> Test set loss: -117.2815
====> Validation set loss: -118.4875
Train Epoch: 4 [0/50000 (0%)]	Loss: -115.504990
Train Epoc

In [0]:
#2.1 reporting for each of the mini-batches to return to dimension[M,]
model.eval()
for batch_index, data in enumerate(valid_loader):
    data = data.to(device)
    reconstruction_batch, mu, log_variance = model(data)

    loss= model.evaluate_IWAE_within_one_mini_batch(200,data,reconstruction_batch , mu , log_variance) 
    #t_loss+=loss.item()
    #Mini_batch_evaluations.append(loss)    
    
    
    if batch_index % 200 == 0:
        print('Size of returned IWAE values for the mini-batch :',
        loss.size() )
        print('IWAE likelihood values within the mini-batch :',
        loss )
        


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.UpsamplingBilinear2d is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Size of returned IWAE values for the mini-batch : torch.Size([128])
IWAE likelihood values within the mini-batch : tensor([ -85.9256,  -51.4764, -108.3658,  -71.6859,  -80.3627,  -98.8165,
        -115.7668,  -64.4819,  -52.1680,  -56.6775,  -91.5293,  -95.2445,
        -107.7856, -109.8123,  -66.3784,  -78.2465,  -54.7036,  -71.5919,
         -94.3613,  -82.3873,  -69.5043,  -81.5381,  -93.9318,  -84.9353,
        -100.1012,  -47.7049,  -82.2398,  -47.6619, -108.3008,  -85.1671,
         -33.5919, -110.5467,  -65.0285, -104.9563,  -47.0480,  -81.4074,
         -97.5503,  -95.4803,  -46.7834, -101.4885,  -92.6153, -110.5799,
         -93.6362,  -90.1419,  -92.1136,  -95.4762,  -95.3591, -117.5358,
        -114.7579,  -77.9413,  -58.0993,  -38.0706, -109.7656,  -67.9145,
         -52.3746,  -79.8533,  -44.0732, -123.8761,  -70.2304,  -76.3815,
         -86.1366,  -63.6862, -112.1142,  -93.1518,  -67.2565,  -75.4955,
         -82.0293,  -88.1159,  -94.9718,  -91.6796,  -71.8233, -102.659

In [0]:


# question 2.2  
#Evaluating likelihood using IWAE on test set 
model.eval()
t_loss=0
for batch_index, data in enumerate(test_loader):
    data = data.to(device)
    reconstruction_batch, mu, log_variance = model(data)

    loss= model.evaluate_IWAE(200,data,reconstruction_batch , mu , log_variance) 
    t_loss+=loss.item()
    
print('Likelihod on whole Test set is : ', t_loss/len(test_loader.dataset)) 

    
#Evaluating likelihood using IWAE on validation set         
model.eval()
t_loss=0
for batch_index, data in enumerate(valid_loader):
    data = data.to(device)
    reconstruction_batch, mu, log_variance = model(data)

    loss= model.evaluate_IWAE(200,data,reconstruction_batch , mu , log_variance) 
    t_loss+=loss.item()
     
print('Likelihod on whole Validation set is : ', t_loss/len(valid_loader.dataset))    

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.UpsamplingBilinear2d is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Likelihod on whole Test set is :  -85.09010629882812
Likelihod on whole Validation set is :  -85.81766625976563
